In [5]:
import pandas as pd
import numpy as np
import locale
locale.setlocale(locale.LC_ALL, '')
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing 

plt.style.use('fivethirtyeight')

# 노트북 안에 그래프 그리기
%matplotlib inline
# 그래프 격자로 숫자 범위가 눈에 잘 띄도록 ggplot 스타일 사용
plt.style.use('ggplot')
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus']=False

# 컬럼을 다 보여주는 함수
from IPython.display import display
pd.options.display.max_columns = None

In [6]:
exam = pd.read_csv('exam.csv')
# 검색어에 결측값이 들어가면 비교불가능하기때문에 제거하고 진행
# 클릭변수를 만들어 제거된 행이 필요한 경우 제거한 값들은 0으로 대체하여 진행
exam = exam[exam['KWD_NM'].notnull()]

In [7]:
exam.head()

,CLNT_ID,SESS_ID,PD_ADD_NM,PD_BRA_NM,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,KWD_NM,SEARCH_CNT
2,4142395,6158159,not set,아베다,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,입생로랑,3.0
3,4142395,6158159,not set,아베다,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,이솝,3.0
4,4142395,6158159,not set,아베다,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,아베다 스무드,1.0
5,4142395,6158159,not set,아베다,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,아베다,23.0
7,4144917,6406509,5개,데코르테,(단독)허니듀 세트(+5만원 상당 샘플 증정),화장품/뷰티케어,스킨케어,스킨케어세트,코스메 데코르테,8.0


In [8]:
# 문장의 유사도 분석
# (KWD_NM과 PD_BRA_NM의 공통 단어) / (KWD_NM의 단어)
# 단어를 2음절씩 나눈이유 : 2음절씩 나누면 문장의 단어를 최대한 세분화해서볼수있음
# 1개는 단어의 의미를 잃기 때문에 안됨

## n_gram
def ngram(s,num):
    res = []
    slen = len(s)#-num+1
    for i in range(slen):
        ss = s[i:i+num]
        res.append(ss)
    return res

def diff_ngram(sa,sb,num):
    a = ngram(sa,num)
    b = ngram(sb,num)
    r = []
    cnt = 0
    for i in a:
        for j in b:
            if i==j:
                cnt +=1
                r.append(i)
    return cnt / len(b)#, r
# 겹치는 단어가 어떤건지 필요하면 r 도 함께 return

In [12]:
print(ngram(exam['KWD_NM'].iloc[16],2))
print(ngram(exam['PD_BRA_NM'].iloc[16],2))

['나이', '이키', '키 ', ' 나', '나시', '시']
['나이', '이키', '키의', '의류', '류']


In [11]:
print(exam['PD_BRA_NM'].iloc[16],exam['KWD_NM'].iloc[16])

나이키의류 나이키 나시


In [68]:
r2= diff_ngram(exam['PD_BRA_NM'].iloc[16],exam['KWD_NM'].iloc[16],2)
print(r2)

(0.3333333333333333, ['나이', '이키'])


In [13]:
r = []
for i in range(len(exam)):
    rr = diff_ngram(exam['PD_BRA_NM'].iloc[i],exam['KWD_NM'].iloc[i],2)
    r.append(rr)

In [14]:
# 검색어와 브랜드의 단어 유사도 변수 생성
exam['brand_search_index'] = r

In [15]:
exam

,CLNT_ID,SESS_ID,PD_ADD_NM,PD_BRA_NM,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,KWD_NM,SEARCH_CNT,brand_search_index
2,4142395,6158159,not set,아베다,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,입생로랑,3.0,"(0.0, [])"
3,4142395,6158159,not set,아베다,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,이솝,3.0,"(0.0, [])"
4,4142395,6158159,not set,아베다,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,아베다 스무드,1.0,"(0.2857142857142857, [아베, 베다])"
5,4142395,6158159,not set,아베다,[단독][6월] 라이트 엘리먼츠 헤어 에센스 세트,퍼스널케어,헤어케어,헤어케어선물세트,아베다,23.0,"(1.0, [아베, 베다, 다])"
7,4144917,6406509,5개,데코르테,(단독)허니듀 세트(+5만원 상당 샘플 증정),화장품/뷰티케어,스킨케어,스킨케어세트,코스메 데코르테,8.0,"(0.5, [데코, 코르, 르테, 테])"
8,4147643,4050743,사이즈옵션04AN868_04 1개,앵글로매니아,[ANGLOMANIA][5종택1] 명품 선글라스 5 - 옵션03.AN868_03,패션잡화,안경/선글라스,여성선글라스,선글라스,1.0,"(0.0, [])"
9,4147643,4050743,사이즈옵션04AN868_04 1개,앵글로매니아,[ANGLOMANIA][5종택1] 명품 선글라스 5 - 옵션03.AN868_03,패션잡화,안경/선글라스,여성선글라스,앵글로매니아,1.0,"(1.0, [앵글, 글로, 로매, 매니, 니아, 아])"
10,4147739,6881895,신발색상블랙BK신발사이즈280,디스커버리,샌드라인 DXSH70831 - 베이지(BG) / 270,아웃도어/레저,등산,등산화,아디다스 반팔티,1.0,"(0.0, [])"
11,4147739,6881895,신발색상블랙BK신발사이즈280,디스커버리,샌드라인 DXSH70831 - 베이지(BG) / 270,아웃도어/레저,등산,등산화,리바이스 티셔츠,1.0,"(0.0, [])"
15,4150259,4098361,색상BLK사이즈105,아레나,남성 일반 4부사각 수영복 ATSMF10BLK - BLK / 095,시즌스포츠,수영/물놀이,남성수영복,와이드팬츠,2.0,"(0.0, [])"


In [ ]:
# PD_ADD_NM, PD_BRA_NM, PD_NM, CLAC1_NM, CLAC2_NM, CLAC3_NM
# vs KWD_NM
# 모두 동일하게 진행하여 검색지수를 생성
# 검색지수 : 고객이 얼마나 목적성을 갖고 검색을 하는가에 대한 지표

#################### 검색과정 ####################

# 대/중/소 분류로 검색을 하는 고객, 특정 브랜드로 검색을 하는 고객, 상품명, (사이즈, 색상)
# 등으로 다양하게 검색이 이루어질것으로 예상
# 6개의 변수와 Search1의 검색어변수의 유사도를 구하여 실제로 구매로 이루어지는 정도(가중치)를
# 파악하여 가중합으로 검색지표를 생성

# 고려해야할 것
# PD_NM과 PD_ADD_NM 의 경우 옵션, 영문 처리를 어떻게 할지?
# 특수문자 제거 후 어떻게 나눠야 할까

# 문제점 
# 반팔 <> 티셔츠 => 같은 의미의 단어지만 검색을 어떻게 했느냐에따라 검색지수가 다르게 나옴
# 문장유사도 분석으로는 이런차이를 모두 확인하기 어려움
# 불완전하지만 다른 지표들을 더 생성해서 보완해야 할듯

# 하지만 문장유사도를 보려면 이방법 밖에 없는듯....
# 연관검색어 등으로 접근하려면 word2vec 사용 고려